In [2]:
from modules.data_utils import *
from modules.db_utils import *

2025-04-16 14:51:45.153 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 14:51:45.156 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 14:51:45.159 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 14:51:45.161 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [14]:
agente = "Daniel Brognaro Penido"

# Connect to the SQLite database
try:
    with sqlite3.connect("DataBase.db") as conn:
        # Query the database for email and phone
        query = """
        SELECT "e-mail", telefone 
        FROM Contatos_Agentes 
        WHERE agente = ?
        """
        df = pd.read_sql_query(query, conn, params=(agente,))
        
        # Check if the query returned any rows
        if df.empty:
            print(f"No results found for agent '{agente}' in the database.")
            
        
        # Extract email and phone from the first row
        email = df.iloc[0]["e-mail"]
        phone = df.iloc[0]["telefone"]
        
except sqlite3.Error as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

In [15]:
email

'daniel.penido@cemig.com.br'

In [16]:
phone

'(31)99647-6429'